<a href="https://colab.research.google.com/github/Barboca/FreeCodeCampBarboca/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# helps in text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping




print(tf.__version__)

     |████████████████████████████████| 403.6MB 43kB/s 
     |████████████████████████████████| 471kB 37.6MB/s 
     |████████████████████████████████| 4.0MB 51.6MB/s 
     |████████████████████████████████| 12.3MB 262kB/s 
     |████████████████████████████████| 4.0MB 56.6MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
2.5.0-dev20210203


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-02-04 00:25:36--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv          [ <=>                ] 349.84K  --.-KB/s    in 0.04s   

2021-02-04 00:25:36 (8.37 MB/s) - ‘train-data.tsv’ saved [358233]

--2021-02-04 00:25:36--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv          [ <=>                ] 115

In [ ]:
train_file = pd.read_table(train_file_path, header=None)
test_file = pd.read_table(test_file_path, header=None)

In [ ]:
#hot-code
train_file[0] = train_file[0].replace(['spam','ham'],[1,0])
test_file[0] = test_file[0].replace(['spam','ham'],[1,0])

In [ ]:
#separando X/Y train_file
X_train = train_file[1]
Y_train = np.array(train_file[0].to_list())

In [ ]:
#separando test_file
X_test = test_file[1]
Y_test = np.array(test_file[0].to_list())

In [ ]:
t = Tokenizer(oov_token='<oov>')
t.fit_on_texts(X_train)

In [ ]:
X_train_t = t.texts_to_sequences(X_train)
X_test_t = t.texts_to_sequences(X_test)

In [ ]:
papped_train = pad_sequences(X_train_t, 50, padding='post', truncating='post')
papped_test = pad_sequences(X_test_t, 50, padding='post', truncating='post')

In [ ]:
vocab_size = len(t.word_index)+1

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 32, input_length=50),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(papped_train, Y_train,epochs=9, validation_split=0.3)

Epoch 1/9
92/92 [==============================] - 4s 31ms/step - loss: 0.4500 - acc: 0.8352 - val_loss: 0.1008 - val_acc: 0.9769
Epoch 2/9
92/92 [==============================] - 2s 26ms/step - loss: 0.0552 - acc: 0.9903 - val_loss: 0.0606 - val_acc: 0.9817
Epoch 3/9
92/92 [==============================] - 2s 26ms/step - loss: 0.0311 - acc: 0.9945 - val_loss: 0.0920 - val_acc: 0.9817
Epoch 4/9
92/92 [==============================] - 2s 26ms/step - loss: 0.0121 - acc: 0.9982 - val_loss: 0.0715 - val_acc: 0.9777
Epoch 5/9
92/92 [==============================] - 2s 26ms/step - loss: 0.0068 - acc: 0.9980 - val_loss: 0.0859 - val_acc: 0.9817
Epoch 6/9
92/92 [==============================] - 2s 27ms/step - loss: 0.0060 - acc: 0.9984 - val_loss: 0.0657 - val_acc: 0.9817
Epoch 7/9
92/92 [==============================] - 2s 27ms/step - loss: 0.0018 - acc: 0.9998 - val_loss: 0.4262 - val_acc: 0.9258
Epoch 8/9
92/92 [==============================] - 2s 26ms/step - loss: 0.0212 - acc: 0.99

In [ ]:
test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

In [ ]:
def previsao(text):
  for i in text:
    texto = t.texts_to_sequences(text)
    texto_pad = pad_sequences(texto,50,padding='post',truncating='post')
    previsao = model.predict(texto_pad)
    return np.argmax(previsao)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(predict_msg):
 texto = t.texts_to_sequences(predict_msg)
 texto_pad = pad_sequences(texto,50,padding='post',truncating='post')
 previsao = model.predict(texto_pad)
 return [np.argmax(previsao), ('ham' if np.argmax(previsao) < 25 else 'spam')]
  

In [ ]:
predict_message("how are you doing today")

[18, 'ham']

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
